In [46]:
import os

In [47]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
from dotenv import load_dotenv
load_dotenv()

True

Load the model and test it

In [49]:
from langchain_core.messages import HumanMessage, SystemMessage

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [51]:
model = ChatGoogleGenerativeAI(model ="gemini-pro", convert_system_message_to_human=True)

In [52]:
message = [SystemMessage(content = "Hi, you are nice bot."), HumanMessage(content ="Hi, how are you?")]

In [53]:
model.invoke(message)

AIMessage(content='I am a virtual assistant, and I am programmed to be helpful and informative. I am not a bot. \n\nHow can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-789820ac-f485-4b1f-a438-f2b24c0d8485-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45})

In [54]:
message2 =[SystemMessage(content = "Hi! You are a nice bot and you answer everything in brief."), HumanMessage(content ="Tell me about red dead redemption.")]

Use Output Parser now

In [55]:
from langchain_core.output_parsers import StrOutputParser

In [56]:
parser = StrOutputParser()

In [57]:
response = model.invoke(message)

In [58]:
parser.invoke(response)

'Thank you for the compliment. I am a virtual assistant, not a bot. I am designed to help people with a variety of tasks, such as answering questions, providing information, and completing tasks.\n\nI am doing well. I am always happy to help people. How can I help you today?'

In [59]:
response2 = model.invoke(message2)

In [60]:
parser.invoke(response2)

'Red Dead Redemption is a Western-themed action-adventure game developed by Rockstar Games. It was released in 2010 for the PlayStation 3 and Xbox 360. The game is set in the American Old West in 1911 and follows the story of John Marston, a former outlaw who is forced to hunt down his former gang members by the government.\n\nRed Dead Redemption received critical acclaim upon release, with praise for its open world, story, characters, and gameplay. It was a commercial success, selling over 15 million copies worldwide. A sequel, Red Dead Redemption 2, was released in 2018.'

Chaining with LCEL(Langchain Expression Language)

Added two components

In [61]:
chain = model | parser 

In [62]:
chain.invoke(message)

'I am a virtual assistant, and I am programmed to be helpful and informative. I am always happy to help you with any questions or tasks you may have.\n\nHow are you doing today?'

In [63]:
chain.invoke(message2)

"Red Dead Redemption is a 2010 action-adventure game developed by Rockstar Games. It is the second installment in the Red Dead series, and a prequel to the 2004 game Red Dead Revolver. The game follows the story of John Marston, a former outlaw who agrees to hunt down his former gang members in order to save his family.\n\nRed Dead Redemption is set in the American Old West in 1911. The game's open world environment allows players to explore the vast and varied landscapes of the West, from the arid deserts of New Austin to the lush forests of West Elizabeth. The game features a variety of activities, including hunting, fishing, gambling, and duels.\n\nRed Dead Redemption was a critical and commercial success. It received widespread praise for its open world environment, story, and characters. The game was also a commercial success, selling over 15 million copies worldwide."

Start Prompting

In [64]:
from langchain_core.prompts import ChatPromptTemplate

In [65]:
system = "translate the following into {language}"
user =  "I am hungry"

In [66]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [67]:
prompt = prompt_template.invoke({"language":"french", "text":"I am learning something"})

In [68]:
model.invoke(prompt)

AIMessage(content="J'apprends quelque chose", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0ddd39c8-a4b5-450f-b7a1-d3e3644ba9d7-0', usage_metadata={'input_tokens': 10, 'output_tokens': 6, 'total_tokens': 16})

In [69]:
prompt.to_messages()

[SystemMessage(content='translate the following into french'),
 HumanMessage(content='I am learning something')]

In [70]:
chain.invoke(prompt)

"J'apprends quelque chose"

Chaining Prompts 

In [71]:
chain = prompt_template | model | parser 

In [72]:
chain.invoke({"language":"Bangla", "text":"I am learning something"})

'আমি কিছু শিখছি'

Agents

In [73]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [74]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [75]:
memory = MemorySaver()

In [76]:
search=TavilySearchResults( max_results=2)

In [77]:
tools = [search]

In [78]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [79]:
config = {"configurable": {"thread_id": "abc123"}}

In [80]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im samin! and I am learning langchain")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is LangChain?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-7d36e104-5ecf-4ade-a54f-939e8e866be4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is LangChain?'}, 'id': '57154d3b-3b0c-4290-890d-94161c025c75', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 24, 'total_tokens': 109})]}}
----
{'tools': {'messages': [ToolMessag

In [81]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live? Print nothing if you can not answer it like empty")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='I cannot fulfill this request. I do not have the ability to access your location or weather data.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-67ce5979-28e8-44aa-8d3b-ebf37059d7d3-0', usage_metadata={'input_tokens': 833, 'output_tokens': 20, 'total_tokens': 853})]}}
----
